# Stock Price Prediction

## Main Goals
- Predict the future price of a stock using its historical data.
- Create lag features, rolling averages, and moving window statistics to capture time-series patterns
- Apply feature scaling (standardization) to prepare data for modeling.

### Context
Financial markets are a cornerstone of the global economy, and predicting stock price movements is a notoriously challenging and highly sought-after goal for investors and financial institutions. Stock prices are not random; they are a form of time-series data where future values are dependent on past values, trends, and volatility. In the field of data science, time-series analysis provides a framework for tackling this problem. By engineering features that capture historical patterns, such as lag features (past prices) and rolling statistics (moving averages), we can transform a sequence of prices into a format suitable for machine learning. This project leverages historical price data for the S&P 500 ETF (SPY) to build a forecasting model, providing a hands-on introduction to time-series feature engineering and prediction.

## 1. Loading in the Data
For this project, we will use the yfinance Python library to programmatically download historical stock price data for the S&P 500 ETF (ticker: SPY). Instead of manually downloading a file, we will write a short script to fetch this data and save it locally as spy_data.csv. his is a common and reproducible method in data science. If you haven't installed the library, you can do so by running pip install yfinance in your terminal. This tool allows us to pull clean, up-to-date data directly from Yahoo Finance.

We will write a short script to download the complete historical data for the S&P 500 ETF (ticker: SPY) and load it directly into a pandas DataFrame. This requires us to import both the yfinance and pandas libraries at the start of our notebook. We'll also take the liberty of saving this data to a csv so as to not have to download the data each time we need to run the code. The following code will fetch the data and prepare it for our use.

In [1]:
#Import necessary libraries
import yfinance as yf
import pandas as pd

#Get the data for the S&P500. 
#It uses the symbol 'SPY', which is the short code for the S&P 500 ETF.
#ticker is short for ticker symbol, which is a unique identifier for a stock.
ticker_data = yf.Ticker('SPY')

#Get the historical prices
#We'll get all available data from the start date until today.
#period="max" gets all the data.
df = ticker_data.history(period="max")

#This saves the downloaded data locally, so you don't have to re-download
#it every time you run your script.
#file_path = 'spy_data.csv'
#df.to_csv(file_path)

#df = pd.read_csv('spy_data.csv', index_col='Date', parse_dates=True)

Once the data is loaded into the spy_df DataFrame, we will display the first few rows to ensure it loaded correctly and to inspect the data's structure. It's important to understand what each column provided by the data source represents, such as Open, High, Low, and Close, which we will use as the basis for our predictions.

In [2]:
#Display the data
display(df)

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
Date,,,,,,,,
1993-01-29 00:00:00-05:00,24.397778,24.397778,24.276396,24.380438,1003200,0.0,0.0,0.0
1993-02-01 00:00:00-05:00,24.397784,24.553846,24.397784,24.553846,480500,0.0,0.0,0.0
1993-02-02 00:00:00-05:00,24.536512,24.623213,24.484491,24.605873,201300,0.0,0.0,0.0
1993-02-03 00:00:00-05:00,24.640549,24.883313,24.623208,24.865973,529400,0.0,0.0,0.0
1993-02-04 00:00:00-05:00,24.952661,25.022022,24.675216,24.970001,531500,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
2025-06-24 00:00:00-04:00,604.330017,607.849976,603.409973,606.780029,67735300,0.0,0.0,0.0
2025-06-25 00:00:00-04:00,607.909973,608.609985,605.539978,607.119995,62114800,0.0,0.0,0.0
2025-06-26 00:00:00-04:00,608.989990,612.309998,608.369995,611.869995,78548400,0.0,0.0,0.0


Here we can see that we were able to import a lot of important data from the yfinance library. We can see we have the price fluctuations throughout the day (through open, high, low, and close), the number of shares in the S&P 500 shared that day (volume), as well as additional information, that is seemingly empty right now. While it doesn't seem like we have enough information to create our models with, please note that stock prices like these tend to have momentum from the previous day(s), and there are still patterns that we can use to predict the upcoming days.

## 2. Preprocessing

Having viewed our data, it's time to start cleaning up our data, and engineer features that will help whatever model we use predict our target. For now, let's start off by cleaning up our data a bit.

### Handling Missing Values. 
As with most projects, we can start by searching for null values, as well as filtering out data that may not be useful to us. Using pandas functions, let's see which features have null entires.

In [3]:
#check for null values
print(df.isnull().sum())

Open             0
High             0
Low              0
Close            0
Volume           0
Dividends        0
Stock Splits     0
Capital Gains    0
dtype: int64


While we fortunately do not have to deal with any null data, we can remember from earlier that our Dividends, Stock Splits, and Capital Gains features were seemingly empty. Let's double check and make sure there are values in these features that aren't just 0.

In [4]:
#Check if there are any values in the Dividends, Stock Splits, and Capital Gains features
print("Total Dividends:", df['Dividends'].sum())
print("Total Stock Splits:", df['Stock Splits'].sum())
print("Total Capital Gains:", df['Capital Gains'].sum())


Total Dividends: 103.96099999999998
Total Stock Splits: 0.0
Total Capital Gains: 0.0


We can see here that we have no stock splits or capital gains, so we know we can drop these features. Before doing so, let's check how many rows actually contain any kind of information for the dividends

In [5]:

# You can also just filter:
print((df['Dividends'] > 0).sum())


131


We see that we only have 131 rows with dividends out of 8159 rows. While it's clear that we want to drop the Stock Splits and Capital Gains features, you might be divided on whether to drop the Dividends feature. However, due to it being sparse in the dataset and potentially being a form of data leakage (as the closing price tends to drop on days dividends are sent out.), we'll go ahead and remove this feature alongside the other two.

In [6]:
#Remove the Dividends, Stock Splits, and Capital Gains columns
df = df.drop(columns=['Dividends', 'Stock Splits', 'Capital Gains'])

### Defining our Target

The next thing we can do is to construct the target variable that our model will learn to predict. While the dataset provides multiple prices for each day (Open, High, Low, Close), we must select a single, consistent value to serve as our prediction goal. For this purpose, we will use the Close price, as it is the standard benchmark that reflects the market's final consensus on the stock's value after a full day of trading. To frame this as a predictive task, we will create a new 'Target' column by shifting the Close price column backwards by one day. This operation effectively sets the target for any given row to be the actual closing price of the following trading day, establishing the direct relationship our model needs to learn. This will naturally result in a NaN value for the final row in our dataset, as its target price lies in the future, and this row will subsequently be removed.

In [7]:
##Defining our Target and dropping the last row with NaN value
df['target'] = df['Close'].shift(-1)
df.dropna(inplace=True)

### Handling the Date

Another thing we have to take care of is the Date feature. As of right now, it's likely stored as a string. While in a useful format, we can't make the most of it yet. Using pandas' .to_datetime function, we can store it as a special datetime object, which will help us with feature engineering later on, as well accessing rows based on the date. 

As such, we'll convert the function to datetime. Note that the Date isn't actually a feature right now, but the dataframe's index. This means we'll have to augument the index as opposed to a 'Date' feature.

In [9]:
#Convert the 'Date', or index from a string into a datetime object.
#Pandas is smart enough to understand this specific format without any extra help.
#utc=True ensures that the datetime is in UTC format.
#This is useful for consistency, especially if you're working with data from different time zones.
df.index = pd.to_datetime(df.index, utc=True)

#Let's verify the result. The Data type of the index should now be 'datetime'.
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8159 entries, 1993-01-29 05:00:00+00:00 to 2025-06-27 04:00:00+00:00
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    8159 non-null   float64
 1   High    8159 non-null   float64
 2   Low     8159 non-null   float64
 3   Close   8159 non-null   float64
 4   Volume  8159 non-null   int64  
 5   target  8159 non-null   float64
dtypes: float64(5), int64(1)
memory usage: 446.2 KB


## 3. Feature Engineering
As we move into the final stage of our preprocessing for now, we will focus on feature engineering. This is the process of creating new, predictive features from our existing data, which is absolutely essential for any time-series forecasting model. Our goal is to transform our single column of historical prices into a rich dataset where each row contains a wealth of context that our model can use to make a prediction. To do this, we will craft features that describe the stock's recent behavior, including its momentum and volatility, giving our model the historical perspective it needs.

Specifically, we will engineer two main types of features. First, we will create lag features, which are simply the price or return values from previous days (such as the price from 1 day ago, 3 days ago, etc.). This directly feeds the model information about recent price action. Second, we will generate rolling window statistics, such as moving averages and standard deviations, over various periods (like 5, 30, and 200 days). A moving average helps the model identify the underlying trend, while a rolling standard deviation provides a crucial measure of recent volatility. 

We will accomplish this using powerful pandas functions like `.shift()` to create the lags and `.rolling()` to compute the window-based statistics, effectively turning our time-series sequence into a structured dataset ready for machine learning.

In [10]:
#It's often better to use returns (percentage change) for features, as they are more stationary.
#Let's create a new column for daily returns, being the percentage change of the 'Close' price.
df['daily_return'] = df['Close'].pct_change()

#Lag Features (based on daily returns)
#We shift the daily_return to use past information to predict the future.
for lag in [1, 2, 3, 5, 10]: #amount of days
    df[f'lag_return_{lag}'] = df['daily_return'].shift(lag)

#Rolling Window Features
#We calculate rolling statistics on the 'Close' price.
#The .shift(1) ensures we are not using the current day's price to create features for itself (lookahead bias).
for window in [5, 10, 30, 90, 200]: # amount of days
    # Rolling Mean (Moving Average)
    df[f'moving_avg_{window}'] = df['Close'].rolling(window=window).mean().shift(1)
    
    # Rolling Standard Deviation (Volatility)
    df[f'volatility_{window}'] = df['Close'].rolling(window=window).std().shift(1)

While we're at it, we also want to go ahead and take advantage of our other features and create features off of those.  We can calculate the range of prices by subtracting high and low for one day, and we can calculate momentum by subtracting the opening prices from the closing prices. We'll also using the .rolling() function to get the average volume over thirty days, and turn that into a feature as well.

In [11]:

#These features capture intra-day dynamics and market activity.
#We shift them by 1 to avoid lookahead bias (we don't know the range until the end of the day).
#Intra-day range
df['intra_day_range'] = (df['High'] - df['Low']).shift(1)

#Intra-day momentum
df['intra_day_momentum'] = (df['Close'] - df['Open']).shift(1)

#Rolling average of volume
df['moving_avg_volume_30'] = df['Volume'].rolling(window=30).mean().shift(1)

#The target for the last day is NaN, and the first N rows (N=largest window)
#will have NaNs from the rolling features. We must drop them.
df.dropna(inplace=True)

#Display the DataFrame shape after creating features and dropping NaNs
print(f"DataFrame shape after creating features and dropping NaNs: {df.shape}")

DataFrame shape after creating features and dropping NaNs: (7959, 25)


With that, we have engineered all the features we need, we're ready to move on to our test train split

## 4. Chronological Test Train Split
While we would typically import the test_train_split function from Sklearn, things aren't as simple this time. We cannot use the standard train_test_split function because it shuffles the data randomly. In forecasting, that would be like using information from the future to predict the past, which is impossible in the real world.

Instead, we must split our data chronologically. We will pick a specific date and use all the data before that date for training our model, and all the data after that date for testing it. This perfectly simulates a real-world scenario where you train a model on historical data and then use it to make predictions on new, unseen data.

Just as usual however, we'll start with separting our features from our target, and then try to create a 80%-20% split so that we have 80% of our data used for training, and 20% used for testing. 

In [12]:
#Calculate the index of the split date at around the 80% mark
split_index = int(len(df) * 0.8)

#Get the actual split date
split_date = df.index[split_index]

#Separate features and target
X = df.drop(columns=['target'])
y = df['target']

#Create a chronological split
X_train = X[X.index < split_date]
X_test = X[X.index >= split_date]
y_train = y[y.index < split_date]
y_test = y[y.index >= split_date]

## 5. Feature Scaling 

Although we had split our data differently than normal, it's all we needed in order to move on to scaling the features. This is because we can still scale the data based on the training data, and then tranform both training and testing data to avoid data leakage. We scale the data so that larger values or features with larger values don't get prioritized by the model. For example, the values in the Volume features are simply much larger by default, so without feature scaling, they'll be treated with more importance in the model.

As such, we'll import the standard scaler from Sklearn, fit it on the training data, and then transform both training and testing sets.

In [13]:
#import standard scaler
from sklearn.preprocessing import StandardScaler

#Initialize the StandardScaler
scaler = StandardScaler()

#Fit the scaler on the training data and transform both train and test sets
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## 6. Building and Training our Model

With our preprocessing complete, we can finally move on to building our model. For this project, we'll be using a linear regression. Being quick, easy to interpret, and is well suited to predicting continous values such as the prices in the stock market. 

As such, we'll import the linear regression module from sklearn, and fit it on the training data.

In [14]:
#Import the linear regression module from sklearn
from sklearn.linear_model import LinearRegression

#Initialize the Linear Regression model
model = LinearRegression()

#Fit the model on the training data
model.fit(X_train_scaled, y_train)

LinearRegression()

## 7. Evaluating our Model 

With our model trained on our training data, it's time to evaluate it. We'll be importing metrics of success from Sklearn and analyzing the results from there. These metrics are

- Root Mean Squared Error (RMSE) – Measures how far, on average, the model's predictions are from the true values. Heavily penalizes larger errors, and is useful when large mistakes are extra costly.

- Mean Absolute Error (MAE) – Averages all absolute differences between prediction and reality. Easy to interpret and less sensitive to outliers. The lower, the better.

- R² Score (Coefficient of Determination) – Tells us how much of the variance in stock prices is explained by our model. The closer to 1, the better.

Let's go ahead with our evaulation.

In [17]:
#Import metrics for evaluation
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

#Make predictions on the test set
y_pred = model.predict(X_test_scaled)

#Calculate evaluation metrics
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

#Display the evaluation metrics
print(f"Root Mean Squared Error: {rmse}")
print(f"R^2 Score: {r2}")
print(f"Mean Absolute Error: {mae}")

Root Mean Squared Error: 4.776039915484373
R^2 Score: 0.9977126573360986
Mean Absolute Error: 3.23980162110866


### Analysis

Let’s take a closer look at how our linear regression model performed in predicting stock prices. Unlike more complex ensemble approaches or models that rely on multiple transformations or feature interactions, we opted for a straightforward linear regression model. This choice was guided by the simplicity of our dataset and the goal of building an interpretable baseline for stock price forecasting.

The results suggest that our model performed remarkably well. We achieved a Root Mean Squared Error (RMSE) of approximately $4.78, which means that, on average, our predictions differ from the actual stock prices by just under five dollars. The Mean Absolute Error (MAE) came in at about $3.24, providing a measure of the typical absolute error between predicted and actual values without disproportionately penalizing larger deviations. Together, these two metrics give us confidence that our model's predictions are not only consistent but reasonably accurate.

Most notably, our model achieved an R² score of 0.9977, indicating that it explains over 99.7% of the variation in stock prices within our dataset. This is an exceptionally strong result for a linear model, and it suggests that the linear relationships captured by our features are highly predictive of the target variable. While R² is not the only measure of success, such a high value indicates that very little variance remains unexplained by our model.

Even though we only used a single model and kept the architecture simple, the results speak for themselves. The low error values and high R² score show that linear regression, when paired with relevant and clean features, can still offer powerful predictive capabilities. It's a strong reminder that in many cases, simpler models can be both effective and interpretable, a valuable combination in both academic and practical settings.

### Conclusion
Throughout this project, we carefully guided the SPY dataset through every stage of the time-series forecasting pipeline, from programmatic data acquisition to specialized feature engineering and the final evaluation of our regression model. By handling time-aware data, creating lag and rolling window features to capture trends and volatility, and implementing a leak-proof chronological split, we prepared our data to reveal the patterns that drive short-term price movements.

But the value of this process goes beyond predicting stock prices alone. Building and refining time-series models teaches us how to find predictive signals in sequential data, transforming a simple list of prices into insights that can inform decisions in fields like finance, economics, and inventory management. The methods we used here, such as creating stationary features from non-stationary data and correctly splitting data chronologically, are essential not only for coursework but for tackling real-world forecasting challenges where accuracy and robust methodology matter.

By practicing data preparation, thoughtful time-series feature engineering, and a rigorous evaluation of our model, we have built a solid foundation for tackling future forecasting tasks. These same skills apply across nearly every area of data science, whether we are forecasting sales, predicting patient vitals, or modeling climate data. For now, good work, and give yourself a pat on the back.